In [ ]:
import nibabel as nib
import numpy as np
import scipy
from scipy.signal import convolve
import open3d as o3d
import k3d
import random

data=nib.load('D:\IdeaProjects\IITP_synth\lv-parametric-modelling\whole_brain_structure\Rigid-semantics+csf.nii.gz').get_fdata()
print(data.shape, np.unique(data))

def voxel_2_gridpoint(vox): 
  ## Generate Boundary_pt
  # print(np.sum(vox==0),np.sum(vox!=0))
  kernel =np.ones((2,2,2))
  # Convolve over all three axes in a for loop
  bd_conv = convolve(vox, kernel)
  print(np.unique(bd_conv))
  bd_conv = np.round(bd_conv)
  bd_conv[(bd_conv!=0)&(bd_conv<6)]=1
  bd_conv[bd_conv==8]=0
  
#   print(np.argwhere(bd_conv==1))
  return np.argwhere(bd_conv==1)
 
data_1 = (data==1).astype(int)

data_1_pt = voxel_2_gridpoint((data==4).astype(int)) # WM
data_2_pt = voxel_2_gridpoint((data==5).astype(int)) # GM
data_3_pt = voxel_2_gridpoint((data==6).astype(int)) # GM

plot = k3d.plot()
for i in range(3,18):
  random_color = (random.randint(0, 0xFFFFFF))
  data_1_pt = voxel_2_gridpoint((data==i).astype(int)) # WM

  # K3D 점 군 생성
  point_cloud1 = k3d.points(positions=data_1_pt.flatten().astype(np.float32),
                          point_size=1, color=random_color)  # 녹색 점
  # plot에 점 군 추가
  plot += point_cloud1

# plot 표시
plot.display()

(182, 218, 182) [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18.]
[-1.23860703e-15 -1.11477139e-15 -1.07367285e-15 ...  8.00000000e+00
  8.00000000e+00  8.00000000e+00]
[-2.56639266e-15 -2.45410937e-15 -2.41409088e-15 ...  8.00000000e+00
  8.00000000e+00  8.00000000e+00]
[-4.07012480e-15 -3.98605703e-15 -3.62099161e-15 ...  8.00000000e+00
  8.00000000e+00  8.00000000e+00]
[-2.69479912e-15 -2.52637417e-15 -2.24566593e-15 ...  8.00000000e+00
  8.00000000e+00  8.00000000e+00]
[-1.23860703e-15 -1.11477139e-15 -1.07367285e-15 ...  8.00000000e+00
  8.00000000e+00  8.00000000e+00]
[-2.56639266e-15 -2.45410937e-15 -2.41409088e-15 ...  8.00000000e+00
  8.00000000e+00  8.00000000e+00]
[-4.07012480e-15 -3.98605703e-15 -3.62099161e-15 ...  8.00000000e+00
  8.00000000e+00  8.00000000e+00]
[-2.69479912e-15 -2.24566593e-15 -1.96495769e-15 ...  8.00000000e+00
  8.00000000e+00  8.00000000e+00]
[-2.69479912e-15 -1.79653274e-15 -1.67618787e-15 ...  8.00000000e+00
  8.00000000

Output()

In [29]:
import nibabel as nib
import numpy as np
import scipy
from scipy.signal import convolve
import open3d as o3d
import k3d
import random
import glob
import util
import copy
import pickle

lv_tail = util.read_dict_from_csv("output.csv",flag ="I")

filelist=glob.glob('G:\ADNIz_prep-by-synthseg\CN2\*\Rigid-synseg.nii.gz')
img_shape = (182,218,182)
points =[
    [img_shape[0]/2,50,img_shape[2]/2],
    [img_shape[0]/2,img_shape[1]-50,img_shape[2]/2],
    [img_shape[0]/2,img_shape[1]/2,50],
    [img_shape[0]/2,img_shape[1]/2,img_shape[2]-50],
    [50,img_shape[1]/2,img_shape[2]/2],
    [img_shape[0]-50,img_shape[1]/2,img_shape[2]/2],
]
lines =[
    [0,1],
    [2,3],
    [4,5]
]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(points),
    lines=o3d.utility.Vector2iVector(lines),
)
coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=30, origin=[40,40,40])
coord_list = [coord]
for i in range(len(filelist)//10):
    drawlist = []
    for ind in range(10):
        sub= filelist[i*10+ind].split('\\')[-2]

        if ind==0: print(filelist[i*10+ind].split('\\')[-2])
        aseg = nib.load(filelist[i*10+ind]).get_fdata()
        # print(np.unique(aseg))
        lv = ((aseg==4)|(aseg==5)).astype(int)
        lv_mesh = util.voxel_to_mesh(lv)
        hippo = (aseg==17).astype(int)
        hippo_mesh = util.voxel_to_mesh(hippo)
        amyg = (aseg==18).astype(int)
        amyg_mesh = util.voxel_to_mesh(amyg)
        hippo_mesh.paint_uniform_color([0.3, 0.5, 1.0])
        amyg_mesh.paint_uniform_color([1.0, 0.6, 0.3])
        hippo_vert = np.asarray(hippo_mesh.vertices)
        amyg_vert = np.asarray(amyg_mesh.vertices)
        lv_vert = np.asarray(lv_mesh.vertices)
        
        hippo_vert[:,0] += 40*ind
        amyg_vert[:,0] += 40*ind
        lv_vert[:,0] += 40*ind
        line_set0=copy.deepcopy(line_set)
        line_points = np.asarray(line_set0.points)
        line_points[:,0] += 40*ind
        
        lv_pcd = util.voxel_to_bd_pcd(lv)
        # try:
        #     data = {}
        #     data['label'] = int(lv_tail[sub][1])
        #     data['input'] = np.asarray(lv_pcd.points)
            
        #     with open(rf"tail_training_data\{sub}_{data['label']}.pkl", 'wb') as f:
        #         pickle.dump(data, f)
        # except:
        #     print(sub)
        #     pass
        # drawlist.append(lv_mesh)
        # drawlist.append(hippo_mesh)
        # drawlist.append(amyg_mesh)
        # drawlist.append(line_set0)
        drawlist.append(lv_pcd)
        o3d.visualization.draw_geometries([lv_mesh])
        
    # o3d.visualization.draw_geometries(drawlist+coord_list)

I1000359


KeyboardInterrupt: 

In [9]:
import numpy as np
import os
# 예제 배열
files=glob.glob('G:\ADNIz_prep-by-synthseg\CN2\*\Rigid-synseg.nii.gz')
filelist = []
for file in files:
    id = file.split('\\')[-2]
    print(id)
    filelist.append(id)
print(filelist)
# CSV 파일로 저장
np.savetxt("output.csv", filelist, delimiter=",", fmt="%s")  # fmt="%d"는 정수 형식 지정
print("파일 저장 완료: output.csv")

I1000359
I1000691
I1001084
I1001329
I1001504
I1002374
I1002458
I1002815
I1003294
I1003400
I1003706
I1003831
I1003993
I1004030
I1004681
I1005122
I1005341
I1006005
I1006805
I1008024
I1008180
I1008408
I1008547
I1008726
I1009041
I1009372
I1009810
I1010018
I1010091
I1010650
I1010734
I1011188
I1011216
I1011962
I1012896
I1013527
I1014117
I1014139
I1014209
I1014334
I1014522
I1015812
I1015823
I1016136
I1016587
I1017836
I1018186
I1018201
I1018387
I1018523
I1018889
I1019224
I1019265
I1020036
I1020096
I1020325
I1020355
I1021434
I1021530
I1021751
I1021826
I1021968
I1022057
I10229251
I10229744
I10231043
I10232516
I10232543
I1023542
I1023583
I10236670
I1023753
I10238706
I10238728
I1024116
I1024384
I10245496
I10247281
I10248336
I1025037
I1025741
I1025881
I1027175
I1027315
I1027631
I1027856
I1028011
I1028499
I1029013
I1029029
I1029048
I1029553
I1029584
I1029891
I1031067
I1031290
I1031685
I1032282
I1032324
I1032569
I1033064
I1033265
I1034852
I1036122
I1037228
I1037249
I1037308
I1037394
I1037417
I1037531

In [19]:
from util import*
lv_tail = read_dict_from_csv("output.csv",flag ="I")
print(lv_tail)
print(len(lv_tail.keys()))

{'I1000691': ['I1000691', '1'], 'I1001084': ['I1001084', '0'], 'I1001329': ['I1001329', '0'], 'I1001504': ['I1001504', '0'], 'I1002374': ['I1002374', '1'], 'I1002458': ['I1002458', '0'], 'I1002815': ['I1002815', '0'], 'I1003294': ['I1003294', '0'], 'I1003400': ['I1003400', '1'], 'I1003706': ['I1003706', '0'], 'I1003831': ['I1003831', '1'], 'I1003993': ['I1003993', '0'], 'I1004030': ['I1004030', '1'], 'I1004681': ['I1004681', '1'], 'I1005122': ['I1005122', '0'], 'I1005341': ['I1005341', '1'], 'I1006005': ['I1006005', '0'], 'I1006805': ['I1006805', '1'], 'I1008024': ['I1008024', '0'], 'I1008180': ['I1008180', '0'], 'I1008408': ['I1008408', '0'], 'I1008547': ['I1008547', '0'], 'I1008726': ['I1008726', '0'], 'I1009041': ['I1009041', '0'], 'I1009372': ['I1009372', '1'], 'I1009810': ['I1009810', '0'], 'I1010018': ['I1010018', '0'], 'I1010091': ['I1010091', '0'], 'I1010650': ['I1010650', '0'], 'I1010734': ['I1010734', '1'], 'I1011188': ['I1011188', '1'], 'I1011216': ['I1011216', '0'], 'I10119

In [28]:
print(data['input'].shape)

(6859, 3)


In [36]:
import nibabel as nib
import numpy as np
import scipy
from scipy.signal import convolve
import open3d as o3d
import k3d
import random
import glob
import util
import copy
import pickle

lv_tail = util.read_dict_from_csv("output.csv",flag ="I")

filelist=glob.glob('G:\ADNIz_prep-by-synthseg\CN2\*\Rigid-synseg.nii.gz')
img_shape = (182,218,182)
points =[
    [img_shape[0]/2,50,img_shape[2]/2],
    [img_shape[0]/2,img_shape[1]-50,img_shape[2]/2],
    [img_shape[0]/2,img_shape[1]/2,50],
    [img_shape[0]/2,img_shape[1]/2,img_shape[2]-50],
    [50,img_shape[1]/2,img_shape[2]/2],
    [img_shape[0]-50,img_shape[1]/2,img_shape[2]/2],
]
lines =[
    [0,1],
    [2,3],
    [4,5]
]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(points),
    lines=o3d.utility.Vector2iVector(lines),
)
coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=30, origin=[40,40,40])
coord_list = [coord]
for i in range(len(filelist)):
    drawlist = []
    sub= filelist[i].split('\\')[-2]

    if ind==0: print(filelist[i].split('\\')[-2])
    aseg = nib.load(filelist[i]).get_fdata()
    # print(np.unique(aseg))
    lv = ((aseg==4)|(aseg==5)).astype(int)
    lv_mesh = util.voxel_to_mesh(lv)
    lv_mesh_draw = copy.deepcopy(lv_mesh)
    hippo = (aseg==17).astype(int)
    hippo_mesh = util.voxel_to_mesh(hippo)
    amyg = (aseg==18).astype(int)
    amyg_mesh = util.voxel_to_mesh(amyg)
    hippo_mesh.paint_uniform_color([0.3, 0.5, 1.0])
    amyg_mesh.paint_uniform_color([1.0, 0.6, 0.3])
    hippo_vert = np.asarray(hippo_mesh.vertices)
    amyg_vert = np.asarray(amyg_mesh.vertices)
    lv_vert = np.asarray(lv_mesh.vertices)
    
    hippo_vert[:,1] += 100
    amyg_vert[:,1] += 100
    lv_vert[:,1] += 100
    line_set0=copy.deepcopy(line_set)
    line_points = np.asarray(line_set0.points)
    line_points[:,1] += 100
    
    lv_mesh_tri = np.asarray(lv_mesh.triangles)
    tri0 = lv_mesh_tri.copy()
    lv_mesh_tri[:,0] = tri0[:,2]
    lv_mesh_tri[:,2] = tri0[:,0]
    
    lv_pcd = util.voxel_to_bd_pcd(lv)

    drawlist.append(lv_pcd)
    o3d.visualization.draw_geometries([lv_mesh_draw, lv_mesh, hippo_mesh, amyg_mesh])
    

I1000359
I1000691
I1001084
I1001329
I1001504
I1002374
I1002458
I1002815
I1003294
I1003400
I1003706


KeyboardInterrupt: 